In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.model_selection import train_test_split
import time

from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasAdamOptimizer
from tensorflow_privacy.privacy.analysis.compute_noise_from_budget_lib import compute_noise


from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackType

import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.plotting as plotting

import numpy as np
from scipy import special

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
learning_rate = 15e-5

In [ ]:
def load_mnist():
    """Loads MNIST and preprocesses to combine training and validation data."""
    train, test = tf.keras.datasets.mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    train_data = train_data.reshape((train_data.shape[0], 28, 28, 1))
    test_data = test_data.reshape((test_data.shape[0], 28, 28, 1))

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

    assert train_data.min() == 0.
    assert train_data.max() == 1.
    assert test_data.min() == 0.
    assert test_data.max() == 1.

    return train_data, train_labels, test_data, test_labels

def membership_inference_attack(model, X_train, X_test, y_train, y_test):
    print('Predict on train...')
    logits_train = model.predict(X_train, batch_size=batch_size)
    print('Predict on test...')
    logits_test = model.predict(X_test, batch_size=batch_size)

    print('Apply softmax to get probabilities from logits...')
    prob_train = special.softmax(logits_train, axis=1)
    prob_test = special.softmax(logits_test, axis=1)

    print('Compute losses...')
    cce = tf.keras.backend.categorical_crossentropy
    constant = tf.keras.backend.constant

    loss_train = cce(constant(y_train), constant(prob_train), from_logits=False).numpy()
    loss_test = cce(constant(y_test), constant(prob_test), from_logits=False).numpy()
    
    labels_train = np.argmax(y_train, axis=1)
    labels_test = np.argmax(y_test, axis=1)

    input = AttackInputData(
      logits_train = logits_train,
      logits_test = logits_test,
      loss_train = loss_train,
      loss_test = loss_test,
      labels_train = labels_train,
      labels_test = labels_test
    )

    # Run several attacks for different data slices
    attacks_result = mia.run_attacks(input,
                                     SlicingSpec(
                                         entire_dataset = True,
                                         by_class = True,
                                         by_classification_correctness = True
                                     ),
                                     attack_types = [
                                         AttackType.THRESHOLD_ATTACK,
                                         AttackType.LOGISTIC_REGRESSION,
                                         AttackType.MULTI_LAYERED_PERCEPTRON,
                                         AttackType.RANDOM_FOREST, 
                                         AttackType.K_NEAREST_NEIGHBORS,
                                         AttackType.THRESHOLD_ENTROPY_ATTACK
                                     ])

    # Plot the ROC curve of the best classifier
#     fig = plotting.plot_roc_curve(
#         attacks_result.get_result_with_max_auc().roc_curve)

    # Print a user-friendly summary of the attacks
    print(attacks_result.summary(by_slices = True))
    time.sleep(5)
    return attacks_result.get_result_with_max_auc().get_auc(), attacks_result.get_result_with_max_attacker_advantage().get_attacker_advantage()

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
def create_cnn(dropout=None, regularizer=None):
    input_data = Input(shape = X_train[0].shape)
    x = Reshape(target_shape=(28, 28, 1))(input_data)
    x = Conv2D(filters=12, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    if dropout is not None:
        x = Dropout(dropout)(x)
    output = Dense(10, kernel_regularizer=regularizer)(x)

    model = Model(input_data, output)
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
            
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    return model

def create_dp_cnn(noise_multiplier, l2_norm_clip, microbatches):
    input_data = Input(shape = X_train[0].shape)
    x = Reshape(target_shape=(28, 28, 1))(input_data)
    x = Conv2D(filters=12, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    output = Dense(10)(x)

    model = Model(input_data, output)
    
    optimizer = DPKerasAdamOptimizer(
                            l2_norm_clip=l2_norm_clip,
                            noise_multiplier=noise_multiplier,
                            num_microbatches=microbatches,
                            learning_rate=learning_rate)
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
    
    model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])
    
    return model

In [ ]:
X_train, y_train, X_test, y_test = load_mnist()
X_train_flat, X_test_flat = X_train.reshape(-1, 28*28*1), X_test.reshape(-1, 28*28*1)

In [ ]:
epochs = 50
batch_size = 100
attacks = 5
settings = [
    (None,None),
    (0.25,None),
    (0.50,None),
    (0.75,None),
    (None,'l2'),
    (0.25,'l2'),
    (0.50,'l2'),
    (0.75,'l2'),
]
results_summary = []

for drop, reg in settings:
    # Instantiate network
    model = create_cnn(dropout=drop, regularizer=reg)
    
    # Train network until convergence
    start_time = time.time()
    r = model.fit(X_train, 
                y_train, 
                validation_data=(X_test, y_test),
                epochs=epochs, 
                batch_size=batch_size,
                #callbacks=[callback]
               )
    end_time = time.time()
    time_elapsed = (end_time - start_time)

    # MIA 
    aauc = []
    aadv = []
    for _ in range(attacks):
        auc, adv = membership_inference_attack(model, X_train, X_test, y_train, y_test)
        aauc.append(auc)
        aadv.append(adv)
    mauc = sum(aauc) / attacks
    madv = sum(aadv) / attacks

    # Write result summary
    summ = ', '.join(map(str,[
        len(r.history['loss']), #epochs
        drop,
        reg,
        r.history['loss'][-1], 
        r.history['val_loss'][-1],
        r.history['accuracy'][-1],
        r.history['val_accuracy'][-1],
        time_elapsed,
        mauc,
        madv
    ]))

    results_summary.append(summ)
    print('='*40)
    
print('Epochs, Dropout, Regularizer, Loss, Val loss, Accuracy, Val accuracy, Time, AUC, Advantage')
for r in results_summary:
    print(r)

In [ ]:
# TEST FOR DIFFERENT EPS
results_summary = []

n = X_train.shape[0]
epochs = 50
batch_size = 100
microbatches = 100
#epsilons = [0.1,0.5,1,2,4,8,16,100,1000]
epsilons = np.linspace(0.1, np.log(2), 5) # 5 values from 0.1 to ln2
#delta = 1e-6
delta = 1e-100 # very small delta
min_noise = 1e-100
l2_norm_clip = 2.5
sampling_rate = batch_size / n
attacks = 5

for e in epsilons:
    # Compute noise multiplier from target epsilon
    noise_multiplier = compute_noise(n, batch_size, e, epochs, delta, min_noise)
    
    # Compute epsilon
    orders = [1 + x / 10. for x in range(1, 100)] + list(range(11, 101))
    sampling_probability = batch_size / n
    rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=epochs * n // batch_size,
                    orders=orders)
    eps = get_privacy_spent(orders, rdp, target_delta=delta)
    
    # Instantiate network
    model = create_dp_cnn(noise_multiplier, l2_norm_clip, microbatches)

    # Train network
    start_time = time.time()
    r = model.fit(X_train, 
                 y_train, 
                 validation_data=(X_test, y_test), 
                 epochs=epochs, 
                 batch_size=batch_size,
                # callbacks=[callback]
                )
    end_time = time.time()
    time_elapsed = (end_time - start_time)

    # MIA
    aauc = []
    aadv = []
    for _ in range(attacks):
        auc, adv = membership_inference_attack(model, X_train, X_test, y_train, y_test)
        aauc.append(auc)
        aadv.append(adv)
    mauc = sum(aauc) / attacks
    madv = sum(aadv) / attacks

    # Write result summary
    summ = ', '.join(map(str,[
          len(r.history['loss']),
          e,
          delta,
          l2_norm_clip,
          noise_multiplier,
          sampling_rate,
          eps[0],
          r.history['loss'][-1], 
          r.history['val_loss'][-1],
          r.history['accuracy'][-1],
          r.history['val_accuracy'][-1],
          time_elapsed,
          mauc,
          madv
    ]))
    results_summary.append(summ)
    print('='*40)
    
print('Epochs, Target epsilon, delta, C, Sigma, Sampling rate, Epsilon, Loss, Val loss, Accuracy, Val accuracy, Time, AUC, Advantage')
for r in results_summary:
    print(r)